### **KLUE 텍스트 데이터 전처리 과정**
- 정규표현식으로 한글만 남겨두기
- pykospacing으로 띄어쓰기 교정
- pyhanspell로 맞춤법 교정
- khaiii 품사 태거로 보통 명사, 고유 명상, 성상 관형사, 동사, 형용사만 남겨두기
- 한국어 사전 정의된 불용어 리스트 이용한 불용어 제거

In [ ]:
!git clone https://github.com/kakao/khaiii.git #git clone
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource #리소스 빌드
!cd /content/build && make install
!cd /content/build && make package_python #파이썬 바인딩
!pip install /content/build/package_python

  Created wheel for khaiii: filename=khaiii-0.4-py3-none-any.whl size=22882715 sha256=b429ed905d57a538798c2d9bb0df2e1b60a464d5d47a41c09d5ae8f177841049
  Stored in directory: /tmp/pip-ephem-wheel-cache-57htfpg8/wheels/79/cb/8c/aed91c3bafdd491bf3fcbed5809b53e50a508c6e167bbbeff8
Successfully built khaiii


In [ ]:
import os
import re
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds #SVD
from string import punctuation

#NLP Preprocessing package
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
print('Ready')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Ready


#### 1) 데이터 로드

In [ ]:
import pandas as pd

klue = pd.read_excel('spaced (1).xlsx')
del klue['Unnamed: 0']
del klue['evaluation']
del klue['evaluation_kor']

In [ ]:
klue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   class_name         218 non-null    object 
 1   professor_name     218 non-null    object 
 2   work_load          214 non-null    float64
 3   level              214 non-null    float64
 4   teaching           214 non-null    float64
 5   proud              214 non-null    float64
 6   evaluation_spaced  216 non-null    object 
dtypes: float64(4), object(3)
memory usage: 12.0+ KB


In [ ]:
from khaiii import KhaiiiApi
api = KhaiiiApi()

#### 2) 품사 태깅
: 주요 품사만 불러오기

In [ ]:
#품사 태깅에 사용하게될 품사 태깅
#보통 명사, 고유 명사, 성상 관형사, 동사, 형용사
significant_tags = ['NNG', 'NNP', 'MM', 'VV', 'VA']

In [ ]:
#khaiii 형태소 분석
def tokenize(texts):
    corpus = []
    for sent in texts:
        token = ''
        for word in api.analyze(sent):
            for morph in word.morphs:
                token += morph.lex + ' '
        corpus.append(token)
    return corpus

In [ ]:
tokenize(klue['evaluation_spaced'][:3])

['학점 은 물론 그 어떤 교수 님 보다 잘 주 시 지만 강의 를 통하 여 배우 ㄴ 것 ㄴ 별로 없 는 것 같 습니다 데이터 네트워크 수업 이 지만 딱히 관련 기초 내용 을 배우 진 않 았 고 최신 트 렌 드 머신 러닝 등 을 활용 하 여 문제 를 해결 하 는 방법 을 주로 설명 하 시 었 습니다 학생 과 의 약속 을 잘 지키 시 지 않 고 보강 도 안 올리 시 는 부분 은 실망 스럽 었 습니다 시험 방식 은 두 문제 정도 주관식 으로 내 시 ㅂ니다 문제 상황 을 해결 하 는 방법 을 제시 하 는 방식 이 었 는데 다른 전공 시험 처럼 시간 이 부족 하 지 도 않 았 고 공부 하 였 던 내용 들 을 답안 을 적 으면서 정리 하 ㄹ 수 있 어서 좋 았 습니다 교수 님 이 약간 학생 들 한테 호감 작 하 는 듯 하 ㄴ 전공 보다 는 교양 의 느낌 이 훨씬 강 하 ㅂ니 다 성적 은 쓸데없이 너무 잘 주 시 고 가르치 는 것 ㄴ 넓 고 얕 게 가르치 시 ㅂ니다 호불호 갈리 ㄹ 터 이 ㄴ데 개 ㄴ 적 으로 는 깊 게 배우 고 싶 었 네요 학점 은 진짜 잘 주시 어 요 학점 은 진짜 잘 주 시 는데 뭐 이 ㄴ가 남 는 것 ㄴ 없 네요 학점 은 진짜 잘 주 시 는데 남 는 것 이 학점 이 길 ㄴ 하 니까 학점 은 진 짜 잘 주 시 ㅂ니다 출석 안 하 시 었 고 과 제도 없 었 는데 중간 기말 시험 딱 두 번 만 보 았 습니다 저 는 시험 각각 시간 정도 강의 만 몰아듣 고 시험 치 었 는데 점수 너무 잘 주 시 어서 공부량 도 없이 에 이쁘 ㄹ 받 았 습니다 학점 잘 주 는 꿀강 을 찾 는 분 에게 ㄴ 완벽 하 ㄴ 강의 이 ㅂ니다 갓 중헌 교수 님 께서 는 학부 생들 을 배려 하 시 어서 내용 도 최대한 재미있 게 설명 을 하 여 주 시 고 이론 보다 는 현재 기술 의 추세 등 을 설명 하 여 주 시 ㅂ니다 그러 어도 좋 은 학점 을 받 기 위하 여 어느 정도 공부 를 하 여야 하 겠 지만 다른 학점 전공 에 비하 면 너무나 꿀 이 ㄴ 강의 수업 은 매시간 줌 라이브 로 

In [ ]:
#khaiii 품사 태깅시 NULL값이 있으면 'float object doesn't have float' 에러 뜸
#NULL값 제거
klue.dropna(axis=0, inplace=True)
print(klue['evaluation_spaced'].isnull().values.any())

False


In [ ]:
#khaiii 품사 태깅
def pos_text(texts):
    corpus = []
    for sent in texts:
        pos_tagged = ''
        for word in api.analyze(sent):
            for morph in word.morphs:
                if morph.tag in significant_tags:
                    pos_tagged += morph.lex + '/' + morph.tag + ' '
        corpus.append(pos_tagged.strip())
    return corpus

In [ ]:
tagged_corpus = pos_text(klue['evaluation_spaced'])

In [ ]:
len(tagged_corpus)

214

#### 3) 표제어 추출(stemming)

In [ ]:
p1 = re.compile('[가-힣A-Za-z0-9]+/NN.') #보통 명사, 고유 명사
p2 = re.compile('[가-힣A-Za-z0-9]+/MM') #성상 관형사
p3 = re.compile('[가-힣A-Za-z0-9]+/VV') #동사
p4 = re.compile('[가-힣A-Za-z0-9]+/VA') #형용사

In [ ]:
def stemming_text(text):
    corpus = []
    for sent in text:
        ori_sent = sent
        mached_terms = re.findall(p1, ori_sent) #NN
        for terms in mached_terms:
            ori_terms = terms #original
            modi_terms = '' #modified
            for term in terms.split(' '):
                lemma = term.split('/')[0] #표제어
                tag = term.split('/')[-1] #태깅된 품사
                modi_terms += lemma
            modi_terms += '/NN' #보통 명사/고유 명사
            ori_sent = ori_sent.replace(ori_terms, modi_terms)
        
        mached_terms = re.findall(p2, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            modi_terms += '/MM' #관형사
            ori_sent = ori_sent.replace(ori_terms, modi_terms)

        mached_terms = re.findall(p3, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            if '다' != modi_terms[-1]:
                modi_terms += '다'
            modi_terms += '/VV' #동사
            ori_sent = ori_sent.replace(ori_terms, modi_terms)

        mached_terms = re.findall(p4, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            if '다' != modi_terms[-1]:
                modi_terms += '다'
            modi_terms += '/VA'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)
        corpus.append(ori_sent)
    return corpus

In [ ]:
stemming_corpus = stemming_text(tagged_corpus)

In [ ]:
for i in range(0, 5):
    print(stemming_corpus[i])

학점/NN 그/MM 어떤/MM 교수/NN 주다/VV 강의/NN 통하다/VV 배우다/VV 없다/VA 같다/VA 데이터/NN 네트워크/NN 수업/NN 관련/NN 기초/NN 내용/NN 배우/NN 진/NN 최신/NN 트/NN 렌/NN 드/NN 머신/NN 러닝/NN 활용/NN 문제/NN 해결/NN 방법/NN 설명/NN 학생/NN 약속/NN 지키다/VV 보강/NN 올리다/VV 부분/NN 실망/NN 시험/NN 방식/NN 두/MM 문제/NN 정도/NN 주관식/NN 내다/VV 문제/NN 상황/NN 해결/NN 방법/NN 제시/NN 방식/NN 다른/MM 전공/NN 시험/NN 시간/NN 부족/NN 공부/NN 내용/NN 답안/NN 적다/VV 정리/NN 있다/VV 좋다/VA 교수/NN 학생/NN 호감/NN 작/NN 전공/NN 교양/NN 느낌/NN 강다/VV 하다/VA 성적/NN 주다/VV 가르치다/VV 넓다/VA 얕다/VA 가르치다/VV 호불호/NN 갈리다/VV 개다/VV 깊다/VA 배우/NN 고/NN 학점/NN 주시다/VV 학점/NN 주다/VV 남다/VV 없다/VA 학점/NN 주다/VV 남다/VV 학점/NN 길다/VA 학점/NN 짜/NN 주다/VV 출석/NN 하다/VV 과/NN 제도/NN 없다/VA 중간/NN 기말/NN 시험/NN 두/MM 보다/VV 시험/NN 시간/NN 정도/NN 강의/NN 몰아듣다/VV 시험/NN 치다/VV 점수/NN 주다/VV 공부량/NN 도/NN 이쁘다/VA 받다/VV 학점/NN 주다/VV 꿀강/NN 찾다/VV 완벽/NN 강의/NN 중헌/NN 교수/NN 학부/NN 배려/NN 내용/NN 최대한/NN 재미있다/VA 설명/NN 하다/VV 이론/NN 기술/NN 추세/NN 설명/NN 그러다/VV 좋다/VA 학점/NN 받다/VV 위하다/VV 어느/MM 정도/NN 공부/NN 하다/VV 다른/MM 학점/NN 전공/NN 비하다/VV 꿀/NN 강의/NN 수업/NN 매시간/NN 줌/NN 라이브/NN 진행/NN 교수/NN 끝나다/VV 유튜브/NN 교수/NN 채널/NN 녹화본/NN 

In [ ]:
len(stemming_corpus)

214

In [ ]:
#(2차 전처리)한글만 남겨두고 제거
def text_cleaning(text):
   hangul=re.compile("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]+")
   result=hangul.sub('', text)
   return result

In [ ]:
for sent in stemming_corpus:
  map(text_cleaning, sent)

In [ ]:
#stemming에 한글만 남기고 제거
corpus=[]
for i in range(0, len(stemming_corpus)):
  corpus.append(text_cleaning(stemming_corpus[i]))

clean_corpus = corpus

In [ ]:
#형태소별로 split
corpus=[]
for sent in clean_corpus:
  corpus.append(sent.split())

corpus_comma = corpus

In [ ]:
for i in range(0, 5):
    print(corpus_comma[i])

['학점', '그', '어떤', '교수', '주다', '강의', '통하다', '배우다', '없다', '같다', '데이터', '네트워크', '수업', '관련', '기초', '내용', '배우', '진', '최신', '트', '렌', '드', '머신', '러닝', '활용', '문제', '해결', '방법', '설명', '학생', '약속', '지키다', '보강', '올리다', '부분', '실망', '시험', '방식', '두', '문제', '정도', '주관식', '내다', '문제', '상황', '해결', '방법', '제시', '방식', '다른', '전공', '시험', '시간', '부족', '공부', '내용', '답안', '적다', '정리', '있다', '좋다', '교수', '학생', '호감', '작', '전공', '교양', '느낌', '강다', '하다', '성적', '주다', '가르치다', '넓다', '얕다', '가르치다', '호불호', '갈리다', '개다', '깊다', '배우', '고', '학점', '주시다', '학점', '주다', '남다', '없다', '학점', '주다', '남다', '학점', '길다', '학점', '짜', '주다', '출석', '하다', '과', '제도', '없다', '중간', '기말', '시험', '두', '보다', '시험', '시간', '정도', '강의', '몰아듣다', '시험', '치다', '점수', '주다', '공부량', '도', '이쁘다', '받다', '학점', '주다', '꿀강', '찾다', '완벽', '강의', '중헌', '교수', '학부', '배려', '내용', '최대한', '재미있다', '설명', '하다', '이론', '기술', '추세', '설명', '그러다', '좋다', '학점', '받다', '위하다', '어느', '정도', '공부', '하다', '다른', '학점', '전공', '비하다', '꿀', '강의', '수업', '매시간', '줌', '라이브', '진행', '교수', '끝나다', '유튜브', '교수', '채널', '녹화본',

#### 4) 불용어 제거

In [ ]:
path="stopwords (2).txt"
with open(path, 'rb') as f:
  stopwords=f.readlines()
stopwords=[x.strip() for x in stopwords]

In [ ]:
corpus=[]
for sentence in corpus_comma:
    temp_X = [word for word in sentence if not word in stopwords]
    corpus.append(temp_X)

removed_stopword_corpus = corpus

In [ ]:
for i in range(0, 5):
    print(removed_stopword_corpus[i])

['학점', '그', '어떤', '교수', '주다', '강의', '통하다', '배우다', '없다', '같다', '데이터', '네트워크', '수업', '관련', '기초', '내용', '배우', '진', '최신', '트', '렌', '드', '머신', '러닝', '활용', '문제', '해결', '방법', '설명', '학생', '약속', '지키다', '보강', '올리다', '부분', '실망', '시험', '방식', '두', '문제', '정도', '주관식', '내다', '문제', '상황', '해결', '방법', '제시', '방식', '다른', '전공', '시험', '시간', '부족', '공부', '내용', '답안', '적다', '정리', '있다', '좋다', '교수', '학생', '호감', '작', '전공', '교양', '느낌', '강다', '하다', '성적', '주다', '가르치다', '넓다', '얕다', '가르치다', '호불호', '갈리다', '개다', '깊다', '배우', '고', '학점', '주시다', '학점', '주다', '남다', '없다', '학점', '주다', '남다', '학점', '길다', '학점', '짜', '주다', '출석', '하다', '과', '제도', '없다', '중간', '기말', '시험', '두', '보다', '시험', '시간', '정도', '강의', '몰아듣다', '시험', '치다', '점수', '주다', '공부량', '도', '이쁘다', '받다', '학점', '주다', '꿀강', '찾다', '완벽', '강의', '중헌', '교수', '학부', '배려', '내용', '최대한', '재미있다', '설명', '하다', '이론', '기술', '추세', '설명', '그러다', '좋다', '학점', '받다', '위하다', '어느', '정도', '공부', '하다', '다른', '학점', '전공', '비하다', '꿀', '강의', '수업', '매시간', '줌', '라이브', '진행', '교수', '끝나다', '유튜브', '교수', '채널', '녹화본',

In [ ]:
klue_khaiii = pd.DataFrame(removed_stopword_corpus) 
klue_khaiii.to_csv('klue_khaiii.csv')

In [ ]:
import csv

with open('klue_khaiii_list.csv', 'w', newline='') as f: 
  writer = csv.writer(f)
  for i in range(0, len(removed_stopword_corpus)):
    writer.writerow(removed_stopword_corpus[i]) 